In [26]:
import datasets
from datasets import load_dataset
import numpy as np
import pandas as pd
import utils
from unidecode import unidecode
import os.path
from transformers import AutoTokenizer, AutoModelForTokenClassification
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
import string
import torch


In [9]:
path_to_data="data\\ADE\\"
texts, labels = [], []
with open(os.path.join(path_to_data, 'DRUG-AE.rel')) as f:
    for line in f:
        pubmed_id, text = line.strip().split('|')[:2]
        texts.append(unidecode(text))
        labels.append(1)

with open(os.path.join(path_to_data, 'ADE-NEG.txt')) as f:
    for line in f:
        pubmed_id, neg = line.strip().split(' ')[:2]
        text = ' '.join(line.strip().split(' ')[2:])
        texts.append(unidecode(text))
        labels.append(0)

In [20]:
df=pd.DataFrame(texts)

In [21]:
df=df.drop_duplicates()

In [22]:
df[0][0:-1]

0            Intravenous azithromycin-induced ototoxicity.
1        Immobilization, while Paget's bone disease was...
2        Unaccountable severe hypercalcemia in a patien...
3        METHODS: We report two cases of pseudoporphyri...
5        Naproxen, the most common offender, has been a...
                               ...                        
23510    Starting with the 8th hospital day, the thromb...
23511    At autopsy, the liver was found to be small, s...
23512    Physical exam revealed a patient with aphasia,...
23513    At the time when the leukemia appeared seven o...
23514    The American Society for Regional Anesthesia a...
Name: 0, Length: 20895, dtype: object

In [24]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def normalize_text(text):
    # Tokenize the text
    tokens = tokenizer(text, return_tensors="pt")

    # Get the tokenized text back as a string
    normalized_text = tokenizer.decode(tokens["input_ids"][0], skip_special_tokens=True)

    return normalized_text

# Apply the normalization function to the DataFrame
df[0] = df[0].apply(normalize_text)

print(df)

                                                       0
0        intravenous azithromycin - induced ototoxicity.
1      immobilization, while paget's bone disease was...
2      unaccountable severe hypercalcemia in a patien...
3      methods : we report two cases of pseudoporphyr...
5      naproxen, the most common offender, has been a...
...                                                  ...
23511  at autopsy, the liver was found to be small, s...
23512  physical exam revealed a patient with aphasia,...
23513  at the time when the leukemia appeared seven o...
23514  the american society for regional anesthesia a...
23515  concomitant administration of estradiol result...

[20896 rows x 1 columns]


In [ ]:
train_size = int(0.7*len(texts))
val_size =int(0.2*len(texts))
test_size = len(texts) - train_size - val_size

#Training data  70 %
train= df[:train_size]
#Validation  20 %  
val= df[train_size:train_size+val_size]
#Test 10 %
test= df[train_size+val_size:] 

In [28]:
# Load model directly
from transformers import AutoModel
model_name="emilyalsentzer/Bio_ClinicalBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [41]:
tokenized_inputs = tokenizer(df[0].tolist(), truncation=True, padding=True)


inputs = tokenizer(df[0][1], return_tensors="pt")
with torch.no_grad():
    outputs = model(**inputs)
    
logits = outputs.logits
# Convert logits to predicted labels
predictions = torch.argmax(logits, dim=-1)
predicted_labels = [model.config.id2label[pred.item()] for pred in predictions[0]]

# Extract entity spans
entity_spans = []
start_index = 0
for i, label in enumerate(predicted_labels):
    if label.startswith("B-"):
        entity_span = [text[start_index:i+1]]
        start_index = i + 1
    elif label.startswith("I-"):
        entity_span.append(text[i])
    elif entity_span:
        entity_spans.append(entity_span)

print(entity_spans)


AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'

In [ ]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

tokens=word_tokenize(text[0])

tagged_words=nltk.pos_tag(tokens)
print(tagged_words)

In [ ]:
preprocess_text(text[1])